In [ ]:
import numpy as np 
import pandas as pd
from konlpy.tag import Okt

callcount = 0


def to_noun_sentence(text):
    global callcount
    global twitter

    callcount += 1
    if callcount % 100 == 0:
        print(callcount)
    stopwords = ['질문', '문의', '관련', '그대로', '계속', '답변', '선생님', '관련문의',
            '한지', '자주', '좀', '쪽', '자꾸', '요즘', '몇개', '무조건', '하나요',
            '안해','요', '경우', '최근', '및', '몇', '달', '일반', '전날', '저번',
            '말', '일어나지', '며칠', '먹기', '지난번', '글', '때문', '너', '무',
            '오늘', '시', '잔', '뒤', '지속', '막', '것', '이건', '뭔가', '다시', '그',
                '무슨', '안', '난', '도', '기', '후', '거리', '이', '뭘', '저', '뭐', '답젼',
                '평생', '회복', '반', '감사', '의사', '보험', '학생', '제발', '살짝',
                '느낌', '제', '대해','갑자기','문제', '전','정도', '왜', '거', '가요',
                '의심', '어제', '추천', '를', '지금', '무엇', '내일', '관해', '리', '세',
                 '로', '목적', '그냥', '거의', '고민', '다음', '이틀', '항상', '뭐', '때',
                '요', '가끔', '이후', '혹시', ]
    twitter = Okt()
    nouns = twitter.nouns(text)
    for word in nouns:
        if word in stopwords:
            while word in nouns:
                nouns.remove(word)
    return ' '.join(nouns)

def preprocess_sentence(sentence):
    twitter = Okt()
    nouns = twitter.nouns(sentence)
    for word in nouns:
        if word in stopwords:
            while word in nouns:
                nouns.remove(word)
    nouns.

def csv_to_datasets(filepath, vocab_num, sequencelength=10):
    import numpy as np 
    import pandas as pd
    import tensorflow as tf
    import tensorflow.keras as keras
    from tensorflow.keras.preprocessing.text import Tokenizer
    from tensorflow.keras.preprocessing.sequence import pad_sequences
    from sklearn.model_selection import train_test_split
    from sklearn.utils import class_weight

    df = pd.read_csv(filepath)
    
    df['nounlist'] = df['nouns'].str.split()
    df.dropna(inplace=True)

    from sklearn.model_selection import train_test_split
    x_train_, x_test_, y_train, y_test = train_test_split(
    df['nounlist'], df['label'], 
    test_size=0.2, random_state=1234, 
    stratify=df['label']
    )

    vocab_size = vocab_num
    t = Tokenizer(num_words=vocab_size)
    t.fit_on_texts(df['nounlist'])

    sequence_length = sequencelength
    trunc_type = 'post'
    padding_type = 'post'

    x_train = t.texts_to_sequences(x_train_)
    padded_x_train = pad_sequences(x_train, truncating=trunc_type, padding=padding_type, maxlen=sequence_length)
    
    x_test = t.texts_to_sequences(x_test_)
    padded_x_test = pad_sequences(x_test, truncating=trunc_type, padding=padding_type, maxlen=sequence_length)

    weight = class_weight.compute_class_weight('balanced', np.unique(y_train), y_train)
    weight = {i : weight[i] for i in range(26)}

    y_train = keras.utils.to_categorical(y_train)
    y_test = keras.utils.to_categorical(y_test)



def preprocess_csv(filepath, exportpath, startindex=0, endindex=None):
    df = pd.read_csv(filepath)
    if endindex != None:
        df = df[startindex:endindex]
    
    df.dropna(inplace=True)
    df['symptom'] = df['symptom'].str.replace(pat='([ㄱ-ㅎㅏ-ㅣ]+)', repl=r' ', regex=True)
    df['symptom'] = df['symptom'].str.replace(pat='[^\w\s]', repl=r' ', regex=True)
    
    df['nouns'] = df['symptom'].apply(to_noun_sentence)
    
    #라벨 정수화
    class_to_label = {'DERM': 0, 'GS': 1, 'IP': 2, 'GI':3, 'OPH':4,
                  'NR': 5, 'ENT': 6, 'PSY': 7, 'HEON': 8, 'RHEU': 9,
                  'REHM': 10, 'NS': 11, 'AN': 12, 'DENT': 13, 'PS': 14,
                  'CS': 15, 'INFC': 16, 'OS': 17,
                  'EMR': 18, 'ENDO': 19, 'CA': 20, 'KTM': 21, 'OBGY': 22,
                  'URO': 23, 'ALL': 24, 'NPH': 25,'LAB': 26}
    df['label'] = df['class'].map(class_to_label)
    
    # 2개 컬럼만 남김
    df = df[['nouns', 'label']]
    
    # NaN 삭제
    df.dropna(inplace=True)
    df['label'] = df['label'].astype(int)
    
    df.to_csv(exportpath, encoding='utf-8', index=False)
    print('done')